In [1]:
import tensorflow as tf
import os
from PIL import Image, ImageDraw
import numpy as np
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def download_dataset():
    drive_dir = "/content/drive/MyDrive/viton-plus"
    os.makedirs(drive_dir)
    !kaggle datasets download mohammadalijalili/viton-plus -p {drive_dir}
    !unzip /content/drive/MyDrive/viton-plus/viton-plus.zip -d {drive_dir}/viton-plus

In [5]:
 drive_dir = "/content/drive/MyDrive/viton-plus"

 if not os.path.exists(drive_dir):
    download_dataset()

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/viton-plus/viton-plus/train/pose/012689_0_keypoints.json  
  inflating: /content/drive/MyDrive/viton-plus/viton-plus/train/pose/012690_0_keypoints.json  
  inflating: /content/drive/MyDrive/viton-plus/viton-plus/train/pose/012692_0_keypoints.json  
  inflating: /content/drive/MyDrive/viton-plus/viton-plus/train/pose/012693_0_keypoints.json  
  inflating: /content/drive/MyDrive/viton-plus/viton-plus/train/pose/012695_0_keypoints.json  
  inflating: /content/drive/MyDrive/viton-plus/viton-plus/train/pose/012696_0_keypoints.json  
  inflating: /content/drive/MyDrive/viton-plus/viton-plus/train/pose/012697_0_keypoints.json  
  inflating: /content/drive/MyDrive/viton-plus/viton-plus/train/pose/012698_0_keypoints.json  
  inflating: /content/drive/MyDrive/viton-plus/viton-plus/train/pose/012699_0_keypoints.json  
  inflating: /content/drive/MyDrive/viton-plus/viton-plus/train/pose/012700_0_keypoints.json  

In [35]:
class CPDataset:
    def __init__(self, data_path, stage='GMM', fine_height=256, fine_width=192):
        self.data_path = data_path
        self.stage = stage
        self.fine_height = fine_height
        self.fine_width = fine_width

    def data_generator(self, data_list):
        with open(data_list, 'r') as f:
            lines = f.readlines()

        for line in lines:
            im_name, c_name = line.strip().split()
            yield self._parse_function(im_name, c_name)

    def _parse_function(self, im_name, c_name):
        # Load and process the cloth image
        c = tf.io.read_file(os.path.join(self.data_path, 'cloth', c_name))
        c = tf.image.decode_jpeg(c, channels=3)
        cm = tf.io.read_file(os.path.join(self.data_path, 'cloth-mask', c_name))
        cm = tf.image.decode_jpeg(cm, channels=1)

        # Normalize images
        c = tf.cast(c, tf.float32) / 127.5 - 1
        cm = tf.cast(cm >= 128, tf.float32)

        # Load person image
        im = tf.io.read_file(os.path.join(self.data_path, 'image', im_name))
        im = tf.image.decode_jpeg(im, channels=3)
        im = tf.cast(im, tf.float32) / 127.5 - 1

        # Load and process parsing image
        parse_name = im_name.replace('.jpg', '.png')
        im_parse = tf.io.read_file(os.path.join(self.data_path, 'image-parse-new', parse_name))
        im_parse = tf.image.decode_png(im_parse, channels=1)
        parse_array = tf.cast(im_parse, tf.int32)

        # Load and process mask image
        im_mask = tf.io.read_file(os.path.join(self.data_path, 'image-mask', parse_name))
        im_mask = tf.image.decode_png(im_mask, channels=1)
        mask_array = tf.cast(im_mask, tf.int32)

        # Create parse shape
        parse_shape = tf.cast(mask_array > 0, tf.float32)

        # Select parse head for GMM stage
        if self.stage == 'GMM':
            parse_head = tf.cast(
                tf.equal(parse_array, 1) | tf.equal(parse_array, 4) | tf.equal(parse_array, 13), tf.float32)
        else:
            parse_head = tf.cast(
                tf.equal(parse_array, 1) | tf.equal(parse_array, 2) | tf.equal(parse_array, 4) |
                tf.equal(parse_array, 9) | tf.equal(parse_array, 12) | tf.equal(parse_array, 13) |
                tf.equal(parse_array, 16) | tf.equal(parse_array, 17), tf.float32)

        # Select parse cloth
        parse_cloth = tf.cast(
            tf.equal(parse_array, 5) | tf.equal(parse_array, 6) | tf.equal(parse_array, 7), tf.float32)

        # Resize and process shape
        parse_shape = tf.image.resize(parse_shape, (self.fine_height, self.fine_width))
        shape = tf.expand_dims(parse_shape, axis=-1) * 2 - 1

        # Create agnostic representation
        print(f"Shape of im: {im.shape}")

        # Ensure parse_head_expanded has the correct shape (expand dims if necessary)
        if len(parse_head_expanded.shape) == 3:  # Assuming it's missing the channel dimension
            parse_head_expanded = tf.expand_dims(parse_head_expanded, axis=-1)
        print(f"Shape of parse_head_expanded: {parse_head_expanded.shape}")
        parse_head_expanded = parse_head_expanded = tf.broadcast_to(parse_head_expanded, tf.shape(im))
        print(f"Shape of parse_head_expanded: {parse_head_expanded.shape}")

        # Combine head and original image
        im_h = im * parse_head_expanded - (1 - parse_head_expanded)

        return {'image': im_h, 'cloth': c, 'cloth_mask': cm, 'parse_shape': shape, 'parse_cloth': parse_cloth}

In [36]:
# Create a dataloader using tf.data.Dataset.from_generator
def create_dataloader(dataroot, datamode, stage, data_list, fine_height, fine_width, radius, batch_size, shuffle=False, warp_cloth_available=False):
    dataset_obj = CPDataset(
        data_path=dataroot,
        stage=stage,
        fine_height=fine_height,
        fine_width=fine_width
    )

    dataset = tf.data.Dataset.from_generator(
        lambda: dataset_obj.data_generator(data_list),
        output_signature={
            'image': tf.TensorSpec(shape=(fine_height, fine_width, 3), dtype=tf.float32),
            'cloth': tf.TensorSpec(shape=(fine_height, fine_width, 3), dtype=tf.float32),
            'cloth_mask': tf.TensorSpec(shape=(fine_height, fine_width, 1), dtype=tf.float32),
            'parse_shape': tf.TensorSpec(shape=(fine_height, fine_width, 1), dtype=tf.float32),
            'parse_cloth': tf.TensorSpec(shape=(fine_height, fine_width, 1), dtype=tf.float32)
        }
    )

    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)  # Adjust buffer size as needed

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

In [37]:
# Dictionary with options for creating the dataloader
options = {
    'dataroot': '/content/drive/MyDrive/viton-plus/viton-plus/train',
    'datamode': 'train',
    'stage': 'GMM',
    'data_list': '/content/drive/MyDrive/viton-plus/viton-plus/train_pairs.txt',
    'fine_height': 256,
    'fine_width': 256,
    'radius': 10,
    'batch_size': 16,
    'shuffle': True
}

# Create the dataset using the defined options
dataset = create_dataloader(
    dataroot=options['dataroot'],
    datamode=options['datamode'],
    stage=options['stage'],
    data_list=options['data_list'],
    fine_height=options['fine_height'],
    fine_width=options['fine_width'],
    radius=options['radius'],
    batch_size=options['batch_size'],
    shuffle=options['shuffle'],
    warp_cloth_available=False
)

In [38]:
# Iterate over the dataset and display batch contents
for batch in dataset.take(1):
    print("Batch contents:")
    for key, value in batch.items():
        print(f"{key}: {value.shape}, dtype: {value.dtype}")

    im = batch['image'][0].numpy()
    im = (im + 1) * 127.5  # Convert back to [0, 255] range
    im = im.astype(np.uint8)

    # Display the image
    plt.imshow(im)
    plt.title("Sample Image")
    plt.axis('off')
    plt.show()

Shape of im: (256, 192, 3)


UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} UnboundLocalError: local variable 'parse_head_expanded' referenced before assignment
Traceback (most recent call last):

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "<ipython-input-35-1ca5908582d6>", line 14, in data_generator
    yield self._parse_function(im_name, c_name)

  File "<ipython-input-35-1ca5908582d6>", line 68, in _parse_function
    if len(parse_head_expanded.shape) == 3:  # Assuming it's missing the channel dimension

Unboun